## Work
1. 請比較使用 l1, l1_l2 及不同比例下的訓練結果

In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
train, test = keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 11s 0us/step


In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.regularizers import l1, l2, l1_l2

In [27]:

def build_mlp(input_dim, num_neurons = [512, 256, 128], l1_ratio = 1e-4):
    """Code Here
    建立你的神經網路
    """
    input_layer = keras.layers.InputLayer(input_shape=input_dim)
    for i, n_units in enumerate(num_neurons):
      if i == 0:
        hidden_layer = keras.layers.Dense(units=n_units, activation='relu', name = 'hidden_layer_{}'.format(i+1), 
                                          kernel_regularizer = l1(l1_ratio))(input_layer)
      else:
        hidden_layer = keras.layers.Dense(units=n_units, activation='relu', name = 'hidden_layer_{}'.format(i+1), 
                                          kernel_regularizer = l1(l1_ratio))(hidden_layer)
    output_layer = keras.layers.Dense(units=10, activation='softmax', name = 'output_layer')(hidden_layer)
    model = keras.models.Model(inputs = [input_layer], outputs = [output_layer])
    return model

In [42]:
input_layer = keras.layers.InputLayer(input_shape=100)
# hidden_layer = keras.layers.Dense(units=32, activation='relu', name = 'hidden_layer_{}'.format(1), 
#                                           kernel_regularizer = l2(1e-4))(input_layer)
l2_ratio=1e-4
x = keras.layers.Dense(units=x_train.shape[1:], 
                        activation="relu", 
                        name="hidden_layer"+str(1), 
                       kernel_regularizer=l2(l2_ratio))(input_layer)

TypeError: ignored

In [28]:
"""Code Here
設定超參數
"""
learning_rate = 1e-3
epochs = 50
batch_size = 256
momentum = 0.95
l1_ratio_set = [1e-2, 1e-4, 1e-8, 1e-12]

In [29]:
results = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""
for l1_ratios in l1_ratio_set:
  keras.backend.clear_session()
  print('l1_ratio:{}'.format(l1))
  model = build_mlp(x_train.shape[1:], l1_ratio=l1_ratios)
  optimizer = keras.optimizers.SGD(lr = learning_rate, nesterov=True, momentum=momentum)
  model.compile(loss = 'categorical_crossentropy', metrics = ['accurcy'], optimizer = optimizer)
  model.fit(x_train, y_train, 
            epochs = epochs, 
            batch_size = batch_size, 
            validation_data = [x_test, y_test], 
            shuffle = True)
  training_loss = model.history.hisrtory['loss']
  val_loss = model.history.history['val_loss']
  training_acc = model.history.history['accuracy']
  val_acc = model.history.history['val_accuracy']

  name = 'l1_ratio = {}'.format(l1)
  results[name] = {'training_loss': training_loss, 
                   'val_loss': val_loss, 
                   'training_acc': training_acc, 
                   'val_acc': val_acc}


l1_ratio:0.01


TypeError: ignored

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
"""Code Here
將結果繪出
"""